In [68]:
import numpy as np
import pandas as pd
import nltk
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer ,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

## preprocessing

In [54]:
def remove_special_chars(text):
  # Regular expression to match all characters except alphanumeric (a-z, A-Z, 0-9) and underscore
    text = re.sub(r'\b\S*\.com\S*\b', '', text, flags=re.MULTILINE)
    # Remove links starts with http/https/www
    text = re.sub(r'http\S+\.com\S*|www\S+\.com\S*|https\S+\.com\S*', '', text, flags=re.MULTILINE)
    # Remove words starting with # or @
    text = re.sub(r'(@|#)\w+', '', text)
    # Remove non-alphanumeric characters (except spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Convert to lowercase and remove extra spaces
    cleaned_text = ' '.join(text.lower().split())
    return cleaned_text
def preprocess_text(text):
  text = remove_special_chars(text)
  # Tokenization (sentence -> words)
  tokens = nltk.word_tokenize(text)

  # Lowercase

  # Stop word removal
  stopwords = nltk.corpus.stopwords.words('english')
  tokens = [token for token in tokens if token not in stopwords]

  # Lemmatization
  lemmatizer = nltk.stem.WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(token) for token in tokens]

  return tokens

In [55]:
columns= ["id","Company","Sentiment","tweet"]
df = pd.read_csv('twitter_training.csv',names=columns)
df.head()

,id,Company,Sentiment,tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [56]:
df = df.drop_duplicates()
df = df.dropna()
df.head()

,id,Company,Sentiment,tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [57]:
df['tweet_tokenized'] = df['tweet'].apply(preprocess_text)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71656 entries, 0 to 74681
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               71656 non-null  int64 
 1   Company          71656 non-null  object
 2   Sentiment        71656 non-null  object
 3   tweet            71656 non-null  object
 4   tweet_tokenized  71656 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.3+ MB


In [58]:

le_model = LabelEncoder()
df['Sentiment_encoded'] = le_model.fit_transform(df['Sentiment'])
df.head()

,id,Company,Sentiment,tweet,tweet_tokenized,Sentiment_encoded
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,"[im, getting, borderland, murder]",3
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,"[coming, border, kill]",3
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,"[im, getting, borderland, kill]",3
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,"[im, coming, borderland, murder]",3
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,"[im, getting, borderland, 2, murder]",3


## TF-IDF

In [67]:
df['answer_processed'] = df['tweet_tokenized'].apply(lambda x: ' '.join(x))

# Initialiser le TfidfVectorizer
vectorizer = TfidfVectorizer()

# Appliquer le TF-IDF sur les textes prétraités
tfidf_matrix = vectorizer.fit_transform(df['answer_processed']).toarray()
print((tfidf_matrix))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Bag Of words

In [74]:
vectorizer1 = CountVectorizer()
bow_representation = vectorizer1.fit_transform(df['answer_processed'][:1000]).toarray()
bow_representation

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## Words Embedding

In [59]:

# Assurez-vous que votre colonne 'answers_tokenized' contient les tokens pour chaque réponse
sentences = df['tweet_tokenized'].tolist()
word2vec_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4 )
# 2. Convertir chaque séquence de tokens en vecteurs Word2Vec

def embed_tweet(tweet_tokenized):
    embedded_vectors = []
    for token in tweet_tokenized:
        try:
            embedded_vectors.append(word2vec_model.wv[token])
        except KeyError:
            embedded_vectors.append(np.zeros(word2vec_model.vector_size))

    # Combine token vectors (e.g., average or sum)
    return np.mean(embedded_vectors, axis=0)  # Return a NumPy array

# Appliquer la fonction 'embed_answer' à chaque séquence de tokens
df['tweet_embedded'] = df['tweet_tokenized'].apply(embed_tweet)
df = df.dropna()
embedded_vectors = np.array(df['tweet_embedded'].tolist())



c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [60]:
df.head()

,id,Company,Sentiment,tweet,tweet_tokenized,Sentiment_encoded,tweet_embedded
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,"[im, getting, borderland, murder]",3,"[0.29278553, 0.64283943, -0.3358855, 0.0906849..."
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,"[coming, border, kill]",3,"[0.039381545, 0.67236084, -0.22423358, 0.51550..."
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,"[im, getting, borderland, kill]",3,"[0.3735851, 0.7062751, -0.31598818, 0.16110942..."
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,"[im, coming, borderland, murder]",3,"[0.32961524, 0.8309766, -0.30997965, 0.1279587..."
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,"[im, getting, borderland, 2, murder]",3,"[0.4352592, 0.7261842, -0.3646326, 0.28815502,..."


## Models "Classfication"

In [61]:

y = df['Sentiment_encoded']
X = embedded_vectors


# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [63]:

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred_log_reg = log_reg.predict(X_test)

# Évaluer le modèle
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log_reg))
print(classification_report(y_test, y_pred_log_reg))


Logistic Regression Accuracy: 0.5192829595772033
              precision    recall  f1-score   support

           0       0.44      0.17      0.24      2469
           1       0.54      0.69      0.61      4123
           2       0.48      0.51      0.50      3487
           3       0.54      0.57      0.55      3923

    accuracy                           0.52     14002
   macro avg       0.50      0.48      0.48     14002
weighted avg       0.51      0.52      0.50     14002



In [66]:
from sklearn.svm import SVC

# Créez un modèle SVM
svm_model = SVC(random_state=42)

# Entraînez le modèle SVM
svm_model.fit(X_train, y_train)

# Faites des prédictions sur les données de test
y_pred_svm = svm_model.predict(X_test)

# Calculez la précision
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Accuracy of SVM:", accuracy_svm)


Accuracy of SVM: 0.5537780317097557


In [75]:
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


SVM Accuracy: 0.5537780317097557
              precision    recall  f1-score   support

           0       0.52      0.18      0.26      2469
           1       0.56      0.74      0.64      4123
           2       0.55      0.50      0.53      3487
           3       0.56      0.64      0.60      3923

    accuracy                           0.55     14002
   macro avg       0.55      0.52      0.51     14002
weighted avg       0.55      0.55      0.53     14002



In [64]:
# Entraîner un modèle de Forêt Aléatoire
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred_rf = rf_model.predict(X_test)

# Évaluer le modèle
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.7302528210255678
              precision    recall  f1-score   support

           0       0.81      0.55      0.65      2469
           1       0.71      0.83      0.77      4123
           2       0.73      0.69      0.71      3487
           3       0.72      0.77      0.75      3923

    accuracy                           0.73     14002
   macro avg       0.74      0.71      0.72     14002
weighted avg       0.74      0.73      0.73     14002



## Interpretation 

Comparons les résultats de classification entre la régression logistique, la machine à vecteurs de support (SVM) et la forêt aléatoire.

1. **Régression logistique**:
   - Précision (Accuracy): 0.519
   - Le modèle a une précision de 51.9%, ce qui signifie que près de 52% des prédictions sont correctes.
   - Les scores de précision, rappel (recall) et F1-score varient pour chaque classe. Par exemple, la classe 1 a une précision de 54% et un rappel de 69%.

2. **SVM**:
   - Précision (Accuracy): 0.554
   - La SVM a une précision légèrement meilleure que la régression logistique, avec 55.4%.
   - Les scores de précision, rappel et F1-score varient également pour chaque classe, avec une amélioration notable dans la classe 0.

3. **Random Forest**:
   - Précision (Accuracy): 0.730
   - Le modèle de forêt aléatoire a une précision beaucoup plus élevée de 73%, indiquant une performance globalement meilleure que les deux autres modèles.
   - Les scores de précision, rappel et F1-score pour chaque classe sont également plus élevés que ceux des deux autres modèles.

**Interprétation**:
- La forêt aléatoire surpasse à la fois la régression logistique et la SVM en termes de précision globale ainsi que de scores de précision, rappel et F1-score pour chaque classe.
- Cela suggère que pour ce problème de classification, la forêt aléatoire est le choix préféré en raison de sa meilleure performance.
- Il est important de noter que le choix du modèle dépend également de facteurs tels que la taille des données, la nature des données et les ressources disponibles pour l'entraînement et le déploiement du modèle.